In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

data_path = "../data/"
data = pd.read_csv(data_path + "dengue2.csv")

dengue2.csv는 Aedes 모기에 의해 전염되는 뎅기열과 관련된 자료이다. 
-	ID: subject ID
-	DENGUE: Dengue fever status: 1=yes, 0=no
-	AGE in years
-	MOSNET: Use of mosquito netting: 1=yes, 0=no 
-	SECTOR: Geographical sector in which the subject lived: 1,2,3,4, or 5


In [2]:
data.head()

,ID,AGE,SEX,MOSNET,DENGUE,SCREEN,SES,SECTOR
0,501,16,2,0,0,2,2.0,1
1,502,33,1,0,0,1,2.0,1
2,503,1,1,0,0,3,2.0,1
3,504,35,1,0,0,1,2.0,1
4,505,6,1,0,0,1,2.0,1


1. AGE, MOSNET, SECTOR를 사용하여 DENGUE를 예측하는 로지스틱 회귀모형을 적합하고 적합된 로지스틱회귀식을 쓰시오.

In [3]:
model = smf.glm('DENGUE ~ AGE + MOSNET + SECTOR', data = data,
                family = sm.families.Binomial()).fit()

In [4]:
model.summary() # sector가 변주형 변수이므로 더미화

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 DENGUE   No. Observations:                  196
Model:                            GLM   Df Residuals:                      192
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -108.92
Date:                Sun, 13 Feb 2022   Deviance:                       217.84
Time:                        17:47:43   Pearson chi2:                     189.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.09001
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.5167      0.502     -5.009      0.000      -3.502      -1.532
AGE            0.0245      0.009      2.829      0.005       0.008       0.042
MOSNET        -0.9300      1.226     -0.758      0.448      -3.334       1.474
SECTOR         0.2901      0.127      2.291      0.022       0.042       0.538
==============================================================================
"""

In [5]:
model = smf.glm('DENGUE ~ AGE + MOSNET + C(SECTOR)', data = data,
                family = sm.families.Binomial()).fit() # SECTOR를 범주화

In [6]:
model.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-3.7866,1.038,-3.648,0.000,-5.821,-1.752
C(SECTOR)[T.2],1.5612,1.117,1.397,0.162,-0.629,3.751
C(SECTOR)[T.3],3.0322,1.078,2.812,0.005,0.919,5.146
C(SECTOR)[T.4],2.7510,1.077,2.553,0.011,0.639,4.863
C(SECTOR)[T.5],2.2200,1.072,2.070,0.038,0.118,4.322
AGE,0.0243,0.009,2.679,0.007,0.007,0.042
MOSNET,-0.3335,1.272,-0.262,0.793,-2.826,2.159


2. 각 회귀계수가 유의한지 검정하시오.

####  강의 20분 참고
- Sector와 sector가 동일하다면 모기망을 사용하는 것은 유의하지 않다.
- 나이가 많을 수록 뎅기열에 전염될 확률이 높다
- Sector2는 의미가 없다
- 변수의 어떤 부분이 1이고 0인지 파악 필요

In [7]:
model.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-3.7866,1.038,-3.648,0.000,-5.821,-1.752
C(SECTOR)[T.2],1.5612,1.117,1.397,0.162,-0.629,3.751
C(SECTOR)[T.3],3.0322,1.078,2.812,0.005,0.919,5.146
C(SECTOR)[T.4],2.7510,1.077,2.553,0.011,0.639,4.863
C(SECTOR)[T.5],2.2200,1.072,2.070,0.038,0.118,4.322
AGE,0.0243,0.009,2.679,0.007,0.007,0.042
MOSNET,-0.3335,1.272,-0.262,0.793,-2.826,2.159


3. 나이와 sector가 동일하다면 모기망을 사용하지 않을 때에 비해 사용할 때 뎅기열에 전염될 odds는 몇 % 감소하는가? 

In [8]:
np.exp(-0.3335) # 0.716배

0.7164118986399645

In [9]:
1-np.exp(-0.3335) # 28.4%감소

0.28358810136003554

- 모기망을 사용할 때에 비해 사용하지 않을 때 뎅기열에 전염될 odds는 음수에서 양수로 바뀐다

In [10]:
data['MOSNET_reverse'] = 1-data['MOSNET']

In [11]:
model_reverse = smf.glm('DENGUE ~ AGE + MOSNET_reverse + C(SECTOR)', data = data,
                family = sm.families.Binomial()).fit().summary().tables[1];model_reverse

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-4.1201,1.598,-2.578,0.010,-7.252,-0.988
C(SECTOR)[T.2],1.5612,1.117,1.397,0.162,-0.629,3.751
C(SECTOR)[T.3],3.0322,1.078,2.812,0.005,0.919,5.146
C(SECTOR)[T.4],2.7510,1.077,2.553,0.011,0.639,4.863
C(SECTOR)[T.5],2.2200,1.072,2.070,0.038,0.118,4.322
AGE,0.0243,0.009,2.679,0.007,0.007,0.042
MOSNET_reverse,0.3335,1.272,0.262,0.793,-2.159,2.826


4. 다른 설명변수의 값이 동일하다면 나이가 1살 증가하면 뎅기열에 걸릴 odds가 몇 % 증가하는가? 5살 증가하면 몇 % 증가하는가? 

In [12]:
np.exp(0.0243) # 1살 증가 -> 2.4% 증가

1.0245976510836627

In [13]:
np.exp(0.0243*5) # 1살 증가 -> 12.9% 증가

1.1291893659251577

5. 다른 설명변수의 값이 동일할 때 가장 뎅기열에 걸릴 위험이 높은 sector는 어디인가? 그곳의 위험은 sector 1에 비해 몇 배 높은가?

#### 강의 30분 참고
- sector1이 기준 -> sector1에 대비해서 얼마정도 큰가 
- sector1 대비 sector2는 차이가 없다
- sector1 대비 나머지는 차이가 있다
- 값이 모두 양수이므로 sector1이 가장 안전
- sector1 대비 log(odds)가 3.0322배 커진다

In [14]:
np.exp(3.0322) # 20배 증가

20.742816625139927